<a href="https://colab.research.google.com/github/magicmercedes/maskrcnn-ppe-detection/blob/master/colab_ppe_detection_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Mask-RCNN Starter Model for the PPE (Personal Protective Equipment) Detection**

---

This notebook shows how to train Mask R-CNN on PPE dataset with transfer learning in Google Colab

Using pre-trained COCO weights trained on http://cocodataset.org as in https:/  github.com/matterport/Mask_RCNN/tree/master/samples/balloon

Colab Runtime type: Python3, GPU enabled. 

**(To enable GPU in your notebook, select the following menu options −> Runtime -> Change runtime type "GPU"**


## Install required packages

Install Matterport's Mask-RCNN model and my ppe documents from github.


In [ ]:
%cd

!git clone https://github.com/matterport/Mask_RCNN.git

/root


In [ ]:
%cd ~/Mask_RCNN

!git clone https://github.com/magicmercedes/maskrcnn-ppe-detection.git

/root/Mask_RCNN
Cloning into 'maskrcnn-ppe-detection'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 84 (delta 1), reused 1 (delta 0), pack-reused 78
Unpacking objects: 100% (84/84), done.


In [ ]:
%cd ~/Mask_RCNN

# Edit requirements file for suitible keras version
!sed -i 's/keras>=2.0.8/keras==2.1.5/g' requirements.txt 

!pip install -q PyDrive
!pip install -r requirements.txt

%tensorflow_version 1.15

/root/Mask_RCNN
     |████████████████████████████████| 163kB 4.1MB/s 
     |████████████████████████████████| 204kB 35.5MB/s 


In [ ]:
%cd ~/Mask_RCNN/

%tensorflow_version 1.15
!pip install q keras==2.1.5
!python setup.py install

/root/Mask_RCNN
`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.15`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.
running install
running bdist_egg
running egg_info
creating mask_rcnn.egg-info
writing mask_rcnn.egg-info/PKG-INFO
writing dependency_links to mask_rcnn.egg-info/dependency_links.txt
writing top-level names to mask_rcnn.egg-info/top_level.txt
writing manifest file 'mask_rcnn.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'mask_rcnn.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/mrcnn
copying mrcnn/utils.py -> build/lib/mrcnn
copying mrcnn/config.py -> build/lib/mrcnn
copying mrcnn/model.py -> build/lib/mrcnn
copying mrcnn/__init__.py -> build/lib/mrcnn
copying mrcnn/parallel_model.py -> build/lib/mrcnn
copying mrcnn/visualize.py -> build/lib/mrcnn
creating build/bdist.li

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# Train model

Pretrained weights options are COCO.

In [ ]:
%cd ~/Mask_RCNN/

import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
# Root directory of the project
import shutil
from tensorflow.python.keras.saving import hdf5_format

shutil.move("/root/Mask_RCNN/maskrcnn-ppe-detection/ppe.py", "/root/Mask_RCNN/ppe.py")

ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
#sys.path.append(os.path.join(ROOT_DIR, "samples\\PPE_detection\\"))  # To find local version
import ppe

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs\\custom")
DATA_DIR = os.path.join(ROOT_DIR,"/root/Mask_RCNN/maskrcnn-ppe-detection/dataset")
#DATA_VAL_DIR = os.path.join(ROOT_DIR,"dataset/val")

# Local path to trained weights file
CUSTOM_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(CUSTOM_MODEL_PATH):
    utils.download_trained_weights(CUSTOM_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")
print(ROOT_DIR)

/root/Mask_RCNN


Using TensorFlow backend.


/
... done downloading pretrained model!
/


**Loading the Dataset**

In [ ]:
 # Training dataset.
dataset_train = ppe.ppeDataset()
dataset_train.load_ppe(DATA_DIR,"train")
dataset_train.prepare()

# Validation dataset
dataset_val = ppe.ppeDataset()
dataset_val.load_ppe(DATA_DIR,"val")
dataset_val.prepare()

/root/Mask_RCNN/maskrcnn-ppe-detection/dataset


## Configurations


Mask R-CNN has a load of hyperparameters. I only adjust some of them.


In [ ]:
class InferenceConfig(ppe.ppeConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                17
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

## Model



This section creates a Mask R-CNN model and specifies augmentations to be used.


In [ ]:
model = modellib.MaskRCNN(mode="training", model_dir=MODEL_DIR, config=config)
model.load_weights(CUSTOM_MODEL_PATH, by_name=True,exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])
model.train(dataset_train, dataset_val,learning_rate=config.LEARNING_RATE, epochs=10, layers='heads')

# The model is configured to log losses and save weights at the end of every epoch.
history = model.keras_model.history.history  






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
box_ind is deprecated, use box_indices instead


## Evaluation

In [ ]:
epochs = range(1,len(next(iter(new_history.values())))+1)
pd.DataFrame(new_history, index=epochs)

plt.figure(figsize=(17,5))

plt.subplot(131)
plt.plot(epochs, new_history["loss"], label="Train loss")
plt.plot(epochs, new_history["val_loss"], label="Valid loss")
plt.legend()
plt.subplot(132)
plt.plot(epochs, new_history["mrcnn_class_loss"], label="Train class ce")
plt.plot(epochs, new_history["val_mrcnn_class_loss"], label="Valid class ce")
plt.legend()
plt.subplot(133)
plt.plot(epochs, new_history["mrcnn_bbox_loss"], label="Train box loss")
plt.plot(epochs, new_history["val_mrcnn_bbox_loss"], label="Valid box loss")
plt.legend()

plt.show()

## Run inference on test dataset

In [ ]:
import os
import cv2
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import skimage
import glob
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

import ppe

# Root directory of the project
ROOT_DIR = os.getcwd()

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library

custom_WEIGHTS_PATH = sorted(glob.glob("/logs/*/mask_rcnn_*.h5"))[-1]

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

config = dog.DogConfig()
custom_DIR = os.path.join(ROOT_DIR, "dataset")

class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
#config.display()

# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"

def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax
  
# Load validation dataset
dataset = dog.DogDataset()
dataset.load_dog(custom_DIR, "val")

# Must call before using the dataset
dataset.prepare()

print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)

# load the last model you trained
# weights_path = model.find_last()[1]

# Load weights
print("Loading weights ", custom_WEIGHTS_PATH)
model.load_weights(custom_WEIGHTS_PATH, by_name=True)

from importlib import reload # was constantly changin the visualization, so I decided to reload it instead of notebook
reload(visualize)